In [ ]:
%config IPCompleter.greedy=True
!pip install azureml-pipeline-wrapper==0.1.0.13166565 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13166565 --ignore-installed

In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module, dsl

#ws = Workspace.get(name='jietong-test-westeurope', subscription_id='f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2',
#                   resource_group='studio-v2-test')
ws = Workspace.get(subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', name='kubeflow_ws_1', resource_group='kubeflow-demo')

In [2]:
# Load modules from workspace
#train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Train')
#score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
#eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')
train_module_func = Module.from_yaml(ws, './train-score-eval/mpi_train.yaml')
score_module_func = Module.from_yaml(ws, './train-score-eval/score.yaml')
eval_module_func = Module.from_yaml(ws, './train-score-eval/eval.yaml')

# Load a module from local as anonymous module
#a_module_in_development_func = Module.load_from_yaml(ws, path='/home/code/new_module/spec.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [3]:
help(train_module_func)

Help on function MPI Train:

MPI Train(training_data: 'Training data organized in the torchvision format/structure' = None, maxrundurationseconds: 'MaxRunDurationSeconds' = '', max_epochs: 'Maximum number of epochs for the training' = None, learning_rate: 'Learning rate, default is 0.01' = '0.01')
    A dummy module to show how to describe MPI module with custom module spec.
    
    
    :param training_data: Training data organized in the torchvision format/structure
    :type training_data: ['AnyFile', 'AnyDirectory']
    :param maxrundurationseconds: MaxRunDurationSeconds
    :type maxrundurationseconds: str
    :param max_epochs: Maximum number of epochs for the training
    :type max_epochs: int
    :param learning_rate: Learning rate, default is 0.01. (optional, default value is 0.01.)
    :type learning_rate: float
    
    
    :output model_output: model_output
    :type: model_output: AnyFile



In [4]:
@dsl.pipeline(
    name='A sample pipeline', 
    description='A sample pipeline uses train/scoring/eval', 
    default_compute_target='cpu-lowpri'
)
def mypipeline():
    train = train_module_func(
        training_data=train_data, 
        max_epochs=5, 
        learning_rate=0.02)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)

p = mypipeline()
p.validate()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SupportDetectView()

{'result': 'validation passed', 'errors': []}

In [5]:
run = p.submit(experiment_name='samples')
#run.wait_for_completion()

Submitted PipelineRun 5563b688-c032-4bae-a310-9b20babcd93a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/5563b688-c032-4bae-a310-9b20babcd93a?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1
